# Single model

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import copy
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from IPython.display import display, clear_output
import glob
import cv2

from utils import get_category_annotations, merge_by_largest_overlap

## Execution parameters

In [ ]:
base_dir = '/data/segpc_phase2/aug_images50/training_logs/final_ensemble_models_and_results'
img_dir = '/data_ssd/segpc_phase2/segpc_val/x'

model_code = 'htc_s101_1x'  # Specify model code for which we want to generate result images

export_images_flag = True  # Needed to generate submission file
export_df_flag = True  # Needed to use these predictions in an ensemble

### No need to modify anything below this point

In [ ]:
model_epoch_map = {
    'htc_r50_1x': 'epoch_11',
    'htc_r101_20e': 'epoch_11',
    'htc_s101_1x': 'epoch_2',
    'scnet_r50_20e': 'epoch_13',
    'scnet_r101_20e': 'epoch_12',
    'scnet_s50_1x': 'epoch_5',
    'scnet_s101_1x': 'epoch_3'
}

epoch = model_epoch_map[model_code]

In [ ]:
ann_file = os.path.join(base_dir, 'final_test_set_annotations.json')
segm_result_file = os.path.join(base_dir, f'{model_code}_segpc_val_set_results_{epoch}.segm.json')
output_dir = os.path.join(base_dir, f'single_model_results/{model_code}_{epoch}')

print('Reading COCO results from:', segm_result_file)

In [ ]:
cocoGt = COCO(ann_file)
result_anns = json.load(open(segm_result_file))

nucleus_id = 0
cytoplasm_id = 1
cell_id = 2

initial_score_thold = 0  # Score threshold to directly filter out some nucleus and cytoplasm candidates

In [ ]:
export_flag = export_images_flag

if export_flag:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

all_results_df = pd.DataFrame()

remove_empty_masks=True

for img_id in cocoGt.getImgIds():
    filename = cocoGt.loadImgs([img_id])[0]['file_name']
    orig_img = plt.imread(os.path.join(img_dir, filename))
    print(f'img_id:{img_id}, filename:{filename}')

    cell_anns_df = get_category_annotations(result_anns, img_id, cell_id, remove_empty_masks, initial_score_thold)
    if cell_anns_df is None:
        print(f'NO CELL ANNOTATIONS GENERATED FOR THIS IMAGE')
        continue
    
    nuc_anns_df = get_category_annotations(result_anns, img_id, nucleus_id, remove_empty_masks, initial_score_thold)
    cyt_anns_df = get_category_annotations(result_anns, img_id, cytoplasm_id, remove_empty_masks, initial_score_thold)
        
    if nuc_anns_df is None and cyt_anns_df is None:
        print(f'NO NUCLEUS NOR CYTOPLASM ANNOTATIONS GENERATED FOR THIS IMAGE')
        continue
    
    merged_anns_df = merge_by_largest_overlap(cell_anns_df, nuc_anns_df, cyt_anns_df)
    merged_anns_df['filename'] = filename
    
    all_results_df = pd.concat([all_results_df, merged_anns_df], axis=0, ignore_index=True)
        
    # Generate images with all the instances for visualization
    all_nuc_masks = np.zeros(orig_img.shape[:2])
    for idx, (nuc_mask, cell_mask) in enumerate(zip(merged_anns_df['nuc_mask'], merged_anns_df['mask'])):
        out_nuc_img = 20*(np.logical_and(np.logical_not(nuc_mask), cell_mask)) + 40*nuc_mask
        # Export individual instances
        if export_flag:
            out_nuc_img_path = os.path.join(output_dir, f'{filename[:-4]}_{idx+1}.bmp')
            plt.imsave(out_nuc_img_path, out_nuc_img, cmap='gray', vmin=0, vmax=255)
    

In [ ]:
if export_df_flag:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    all_results_df.drop(['mask', 'nuc_mask', 'cyt_mask'], axis=1).to_pickle(
        os.path.join(output_dir, f'../all_results_df_{model_code}_{epoch}.pkl'))  